In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
#a

In [3]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data3/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data3/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 

dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
dtt =dtuji('tambahan') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
df1= dtlatih('hasilsmote_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('hasilsmote_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('hasilsmote_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('hasilsmote_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('hasilsmote_5') #df5.X,df5.Y,df5.dt

In [4]:
dtg =dtuji('gabung') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt

In [5]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data3/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data3/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 
df= dtlatih('latih_parted')

In [6]:
#df.X

In [7]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


#RF

In [8]:
def rf(n,m,df):
    # Number of trees in random forest
   # n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2500, num = 3)]
    # Number of features to consider at every split
    #max_features = ['auto', None]
    # Maximum number of levels in tree
    #max_depth = [int(x) for x in np.linspace(10, 100, num = 2)]
    #max_depth.append(None)
    # Minimum number of samples required to split a node
    random_grid = {'n_estimators': [n],#[100,500,1000],
                   #'max_features' : max_features,
                   #'max_depth': max_depth,                 
                   #'bootstrap': [True,False],
                   'max_samples': [m]#[1000,2000,3000]
                   }
    rf = BaggingClassifier(bootstrap=True,base_estimator=DecisionTreeClassifier())
    #rf = RandomForestClassifier()
    
    
    rf = GridSearchCV(rf, random_grid, n_jobs=-1, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    rf.fit(df.X, df.Y)
    filename = '../data3/model/rf_{0}n_{1}m_{2}'.format(n,m,df.dt)
    pickle.dump(rf, open(filename, 'wb'))
    return rf

In [9]:
def report_rf(model,n,m,df,d):
    rf = pickle.load(open('../data3/model/{0}_{1}n_{2}m_hasilsmote_{3}'.format(model.__name__,n,m,d), 'rb'))
    pred_x = rf.predict(df.X)
    print('n = {0}'.format(n))
    print('m = {0}'.format(m))
    print('model file : {0}_{1}n_{2}m_hasilsmote_{3}'.format(model.__name__,n,m,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,n,m,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data3/hasil_pred/{0}_{1}n_{2}m_{3}p_{4}_modelhasilsmote{5}.xlsx'.format(model.__name__,n,m,p,data.dt,d))

In [12]:
def fitn3pred(model,n,m,p,datalatih,datauji1,datauji2,datauji3,d):
    md=report_rf(model,n,m,datalatih,d)
    prediksi(md,model,n,m,p,datauji1,d)
    prediksi(md,model,n,m,p,datauji2,d)
    prediksi(md,model,n,m,p,datauji3,d)
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2,d):
    md=report_rf(model,n,m,datalatih,d)
    prediksi(md,model,n,m,p,datauji1,d)
    prediksi(md,model,n,m,p,datauji2,d)
def reportnpred(model,n,m,p,datalatih,datauji1,d):
    md=report_rf(model,n,m,datalatih,d)
    prediksi(md,model,n,m,p,datauji1,d)

In [13]:
reportnpred(rf,100,1000,0.5,df,dtt,1)
reportnpred(rf,100,1000,0.5,df,dtt,2)
reportnpred(rf,100,1000,0.5,df,dtt,3)
reportnpred(rf,100,1000,0.5,df,dtt,4)
reportnpred(rf,100,1000,0.5,df,dtt,5)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_1
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  tambahan  : (0,1)
[[12 12]
 [ 0  0]]
n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_2
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  tambahan  : (0,1)
[[12 12]
 [ 0  0]]
n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_3
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  tambahan  : (0,1)
[[12 12]
 [ 0  0]]
n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_4
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_4 
hasil prediksi  tambahan  : (0,1)
[[12 12]
 [ 0  0]]
n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_5
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_5

In [74]:
fitn2pred(rf,100,1000,0.5,df,dtf,dtl,dtg,1)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_1
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[144  96]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[1755 1293]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  gabung  : (0,1)
[[1845 1363]
 [   0    0]]


In [78]:
fitn2pred(rf,100,1000,0.5,df,dtf,dtl,dtg,2)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_2
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbafika  : (0,1)
[[120 120]
 [  0   0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbalinda  : (0,1)
[[1526 1522]
 [   0    0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  gabung  : (0,1)
[[1606 1602]
 [   0    0]]


In [79]:
fitn2pred(rf,100,1000,0.5,df,dtf,dtl,dtg,3)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_3
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  mbafika  : (0,1)
[[120 120]
 [  0   0]]
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  mbalinda  : (0,1)
[[1563 1485]
 [   0    0]]
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  gabung  : (0,1)
[[1641 1567]
 [   0    0]]


In [80]:
fitn2pred(rf,100,1000,0.5,df,dtf,dtl,dtg,4)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_4
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_4 
hasil prediksi  mbafika  : (0,1)
[[120 120]
 [  0   0]]
threshold 0.5 
model_hasilsmote_4 
hasil prediksi  mbalinda  : (0,1)
[[1530 1518]
 [   0    0]]
threshold 0.5 
model_hasilsmote_4 
hasil prediksi  gabung  : (0,1)
[[1610 1598]
 [   0    0]]


In [81]:
fitn2pred(rf,100,1000,0.5,df,dtf,dtl,dtg,5)

n = 100
m = 1000
model file : rf_100n_1000m_hasilsmote_5
Best parameters found: {'max_samples': 1000, 'n_estimators': 100}
threshold 0.5 
model_hasilsmote_5 
hasil prediksi  mbafika  : (0,1)
[[123 117]
 [  0   0]]
threshold 0.5 
model_hasilsmote_5 
hasil prediksi  mbalinda  : (0,1)
[[1773 1275]
 [   0    0]]
threshold 0.5 
model_hasilsmote_5 
hasil prediksi  gabung  : (0,1)
[[1855 1353]
 [   0    0]]


In [97]:
rf1_100n_1000m=rf(100,1000,df1)
print('done')
rf1_100n_2000m=rf(100,2000,df1)
print('done')
rf1_100n_3000m=rf(100,3000,df1)
print('done')

done
done
done


In [99]:
rf1_500n_1000m=rf(500,1000,df1)
print('done')
rf1_500n_2000m=rf(500,2000,df1)
print('done')
rf1_500n_3000m=rf(500,3000,df1)
print('done')

done
done
done


In [21]:
rf1_300n_1000m=rf(300,1000,df1)
print('done')
rf1_300n_2000m=rf(300,2000,df1)
print('done')
rf1_300n_3000m=rf(300,3000,df1)
print('done')

done
done
done


In [100]:
rf2_100n_1000m=rf(100,1000,df2)
print('done')
rf2_100n_2000m=rf(100,2000,df2)
print('done')
rf2_100n_3000m=rf(100,3000,df2)
print('done')

done
done
done


In [12]:
rf2_500n_1000m=rf(500,1000,df2)
print('done')
rf2_500n_2000m=rf(500,2000,df2)
print('done')
rf2_500n_3000m=rf(500,3000,df2)
print('done')

done
done
done


In [22]:
rf2_300n_1000m=rf(300,1000,df2)
print('done')
rf2_300n_2000m=rf(300,2000,df2)
print('done')
rf2_300n_3000m=rf(300,3000,df2)
print('done')

done
done
done


In [101]:
rf3_100n_1000m=rf(100,1000,df3)
print('done')
rf3_100n_2000m=rf(100,2000,df3)
print('done')
rf3_100n_3000m=rf(100,3000,df3)
print('done')

done
done
done


In [13]:
rf3_500n_1000m=rf(500,1000,df3)
print('done')
rf3_500n_2000m=rf(500,2000,df3)
print('done')
rf3_500n_3000m=rf(500,3000,df3)
print('done')

done
done
done


In [23]:
rf3_300n_1000m=rf(300,1000,df3)
print('done')
rf3_300n_2000m=rf(300,2000,df3)
print('done')
rf3_300n_3000m=rf(300,3000,df3)
print('done')

done
done
done


In [102]:
rf4_100n_1000m=rf(100,1000,df4)
print('done')
rf4_100n_2000m=rf(100,2000,df4)
print('done')
rf4_100n_3000m=rf(100,3000,df4)
print('done')

done
done
done


In [14]:
rf4_500n_1000m=rf(500,1000,df4)
print('done')
rf4_500n_2000m=rf(500,2000,df4)
print('done')
rf4_500n_3000m=rf(500,3000,df4)
print('done')

done
done
done


In [24]:
rf4_300n_1000m=rf(300,1000,df4)
print('done')
rf4_300n_2000m=rf(300,2000,df4)
print('done')
rf4_300n_3000m=rf(300,3000,df4)
print('done')

done
done
done


In [103]:
rf5_100n_1000m=rf(100,1000,df5)
print('done')
rf5_100n_2000m=rf(100,2000,df5)
print('done')
rf5_100n_3000m=rf(100,3000,df5)
print('done')

done
done
done


In [15]:
rf5_500n_1000m=rf(500,1000,df5)
print('done')
rf5_500n_2000m=rf(500,2000,df5)
print('done')
rf5_500n_3000m=rf(500,3000,df5)
print('done')

done
done
done


In [25]:
rf5_300n_1000m=rf(300,1000,df5)
print('done')
rf5_300n_2000m=rf(300,2000,df5)
print('done')
rf5_300n_3000m=rf(300,3000,df5)
print('done')

done
done
done


In [7]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results(n,m,o):
    rf = pickle.load(open('../data3/model/rf_{0}n_{1}m_{2}'.format(n,m,o.dt), 'rb'))
    print('rf_{0}n_{1}m_{2}'.format(n,m,o.dt))
    print('AUC :',rf.cv_results_['mean_test_AUC'])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'])
    print('Precision :',rf.cv_results_['mean_test_Precision'])
    print('Recall :',rf.cv_results_['mean_test_Recall'])

In [119]:
print_cv_results(100,1000,df1)
print_cv_results(100,1000,df2)
print_cv_results(100,1000,df3)
print_cv_results(100,1000,df4)
print_cv_results(100,1000,df5)

rf_100n_1000m_hasilsmote_1
AUC : 
[0.9810828]
Accuracy : 
[0.93956612]
F-measure : 
[0.93612808]
Precision : 
[0.94530839]
Recall : 
[0.93259298]
rf_100n_1000m_hasilsmote_2
AUC : 
[0.98375491]
Accuracy : 
[0.94292355]
F-measure : 
[0.93930902]
Precision : 
[0.95788422]
Recall : 
[0.92691116]
rf_100n_1000m_hasilsmote_3
AUC : 
[0.98135091]
Accuracy : 
[0.94266529]
F-measure : 
[0.93942414]
Precision : 
[0.95357436]
Recall : 
[0.93001033]
rf_100n_1000m_hasilsmote_4
AUC : 
[0.98132323]
Accuracy : 
[0.94124483]
F-measure : 
[0.93786229]
Precision : 
[0.94843416]
Recall : 
[0.93259298]
rf_100n_1000m_hasilsmote_5
AUC : 
[0.98136758]
Accuracy : 
[0.94008264]
F-measure : 
[0.93725345]
Precision : 
[0.94480344]
Recall : 
[0.93543388]


In [120]:
print_cv_results(100,2000,df1)
print_cv_results(100,2000,df2)
print_cv_results(100,2000,df3)
print_cv_results(100,2000,df4)
print_cv_results(100,2000,df5)

rf_100n_2000m_hasilsmote_1
AUC : 
[0.98395963]
Accuracy : 
[0.94085744]
F-measure : 
[0.9357606]
Precision : 
[0.95099396]
Recall : 
[0.9294938]
rf_100n_2000m_hasilsmote_2
AUC : 
[0.98572354]
Accuracy : 
[0.94899277]
F-measure : 
[0.94513875]
Precision : 
[0.96013675]
Recall : 
[0.93698347]
rf_100n_2000m_hasilsmote_3
AUC : 
[0.98237057]
Accuracy : 
[0.94692665]
F-measure : 
[0.94325587]
Precision : 
[0.95594772]
Recall : 
[0.93672521]
rf_100n_2000m_hasilsmote_4
AUC : 
[0.98383916]
Accuracy : 
[0.94395661]
F-measure : 
[0.93930826]
Precision : 
[0.95482097]
Recall : 
[0.93155992]
rf_100n_2000m_hasilsmote_5
AUC : 
[0.98485891]
Accuracy : 
[0.94369835]
F-measure : 
[0.93939525]
Precision : 
[0.95598861]
Recall : 
[0.93130165]


In [121]:
print_cv_results(100,3000,df1)
print_cv_results(100,3000,df2)
print_cv_results(100,3000,df3)
print_cv_results(100,3000,df4)
print_cv_results(100,3000,df5)

rf_100n_3000m_hasilsmote_1
AUC : 
[0.98411358]
Accuracy : 
[0.94447314]
F-measure : 
[0.93866813]
Precision : 
[0.95856471]
Recall : 
[0.92871901]
rf_100n_3000m_hasilsmote_2
AUC : 
[0.98689083]
Accuracy : 
[0.95170455]
F-measure : 
[0.94709847]
Precision : 
[0.96653533]
Recall : 
[0.93595041]
rf_100n_3000m_hasilsmote_3
AUC : 
[0.98305558]
Accuracy : 
[0.94537707]
F-measure : 
[0.94075606]
Precision : 
[0.95636201]
Recall : 
[0.93285124]
rf_100n_3000m_hasilsmote_4
AUC : 
[0.9854953]
Accuracy : 
[0.94770145]
F-measure : 
[0.94196595]
Precision : 
[0.96318143]
Recall : 
[0.93052686]
rf_100n_3000m_hasilsmote_5
AUC : 
[0.98544027]
Accuracy : 
[0.94486054]
F-measure : 
[0.93925956]
Precision : 
[0.96125704]
Recall : 
[0.92846074]


In [ ]:
#500n

In [16]:
print_cv_results(500,1000,df1)
print_cv_results(500,1000,df2)
print_cv_results(500,1000,df3)
print_cv_results(500,1000,df4)
print_cv_results(500,1000,df5)

rf_500n_1000m_hasilsmote_1
AUC : 
[0.98147183]
Accuracy : 
[0.94163223]
F-measure : 
[0.93863991]
Precision : 
[0.94542826]
Recall : 
[0.93698347]
rf_500n_1000m_hasilsmote_2
AUC : 
[0.98470171]
Accuracy : 
[0.94486054]
F-measure : 
[0.94150758]
Precision : 
[0.9561312]
Recall : 
[0.93285124]
rf_500n_1000m_hasilsmote_3
AUC : 
[0.9818981]
Accuracy : 
[0.94576446]
F-measure : 
[0.9427903]
Precision : 
[0.9552095]
Recall : 
[0.93517562]
rf_500n_1000m_hasilsmote_4
AUC : 
[0.98167433]
Accuracy : 
[0.94137397]
F-measure : 
[0.93795796]
Precision : 
[0.94809922]
Recall : 
[0.9331095]
rf_500n_1000m_hasilsmote_5
AUC : 
[0.98220367]
Accuracy : 
[0.94201963]
F-measure : 
[0.93925998]
Precision : 
[0.94678166]
Recall : 
[0.93724174]


In [17]:
print_cv_results(500,2000,df1)
print_cv_results(500,2000,df2)
print_cv_results(500,2000,df3)
print_cv_results(500,2000,df4)
print_cv_results(500,2000,df5)

rf_500n_2000m_hasilsmote_1
AUC : 
[0.98405898]
Accuracy : 
[0.94331095]
F-measure : 
[0.93877492]
Precision : 
[0.95255019]
Recall : 
[0.93285124]
rf_500n_2000m_hasilsmote_2
AUC : 
[0.98687298]
Accuracy : 
[0.9495093]
F-measure : 
[0.94551844]
Precision : 
[0.96089926]
Recall : 
[0.9375]
rf_500n_2000m_hasilsmote_3
AUC : 
[0.98340276]
Accuracy : 
[0.94821798]
F-measure : 
[0.94493067]
Precision : 
[0.95598102]
Recall : 
[0.93930785]
rf_500n_2000m_hasilsmote_4
AUC : 
[0.98372021]
Accuracy : 
[0.9458936]
F-measure : 
[0.94132348]
Precision : 
[0.95618789]
Recall : 
[0.9338843]
rf_500n_2000m_hasilsmote_5
AUC : 
[0.98476416]
Accuracy : 
[0.9458936]
F-measure : 
[0.94177648]
Precision : 
[0.9566221]
Recall : 
[0.93517562]


In [18]:
print_cv_results(500,3000,df1)
print_cv_results(500,3000,df2)
print_cv_results(500,3000,df3)
print_cv_results(500,3000,df4)
print_cv_results(500,3000,df5)

rf_500n_3000m_hasilsmote_1
AUC : 
[0.98465306]
Accuracy : 
[0.94615186]
F-measure : 
[0.94033652]
Precision : 
[0.95974146]
Recall : 
[0.93078512]
rf_500n_3000m_hasilsmote_2
AUC : 
[0.98739115]
Accuracy : 
[0.95157541]
F-measure : 
[0.94698436]
Precision : 
[0.9658243]
Recall : 
[0.93646694]
rf_500n_3000m_hasilsmote_3
AUC : 
[0.98320611]
Accuracy : 
[0.94692665]
F-measure : 
[0.94211875]
Precision : 
[0.95852267]
Recall : 
[0.93362603]
rf_500n_3000m_hasilsmote_4
AUC : 
[0.98502981]
Accuracy : 
[0.94886364]
F-measure : 
[0.94341445]
Precision : 
[0.96431217]
Recall : 
[0.93207645]
rf_500n_3000m_hasilsmote_5
AUC : 
[0.98529475]
Accuracy : 
[0.94395661]
F-measure : 
[0.9382722]
Precision : 
[0.96027855]
Recall : 
[0.92768595]


In [27]:
print_cv_results(300,1000,df1)
print_cv_results(300,1000,df2)
print_cv_results(300,1000,df3)
print_cv_results(300,1000,df4)
print_cv_results(300,1000,df5)

rf_300n_1000m_hasilsmote_1
AUC : [0.98120402]
Accuracy : [0.94059917]
F-measure : [0.93752184]
Precision : [0.94500829]
Recall : [0.93517562]
rf_300n_1000m_hasilsmote_2
AUC : [0.98410388]
Accuracy : [0.94395661]
F-measure : [0.94088782]
Precision : [0.95369562]
Recall : [0.93336777]
rf_300n_1000m_hasilsmote_3
AUC : [0.98162226]
Accuracy : [0.94576446]
F-measure : [0.94287788]
Precision : [0.95560493]
Recall : [0.93465909]
rf_300n_1000m_hasilsmote_4
AUC : [0.98167743]
Accuracy : [0.94266529]
F-measure : [0.93945147]
Precision : [0.94971889]
Recall : [0.93414256]
rf_300n_1000m_hasilsmote_5
AUC : [0.98259717]
Accuracy : [0.9415031]
F-measure : [0.93854936]
Precision : [0.9482732]
Recall : [0.93491736]


In [28]:
print_cv_results(300,2000,df1)
print_cv_results(300,2000,df2)
print_cv_results(300,2000,df3)
print_cv_results(300,2000,df4)
print_cv_results(300,2000,df5)

rf_300n_2000m_hasilsmote_1
AUC : [0.98373448]
Accuracy : [0.94434401]
F-measure : [0.93953597]
Precision : [0.95540913]
Recall : [0.93181818]
rf_300n_2000m_hasilsmote_2
AUC : [0.98671772]
Accuracy : [0.95041322]
F-measure : [0.94656373]
Precision : [0.96210169]
Recall : [0.93801653]
rf_300n_2000m_hasilsmote_3
AUC : [0.9837412]
Accuracy : [0.94847624]
F-measure : [0.94470895]
Precision : [0.95851316]
Recall : [0.93724174]
rf_300n_2000m_hasilsmote_4
AUC : [0.98396295]
Accuracy : [0.94744318]
F-measure : [0.94299574]
Precision : [0.95876207]
Recall : [0.93465909]
rf_300n_2000m_hasilsmote_5
AUC : [0.98477419]
Accuracy : [0.94524793]
F-measure : [0.94097349]
Precision : [0.95722798]
Recall : [0.93336777]


In [29]:
print_cv_results(300,3000,df1)
print_cv_results(300,3000,df2)
print_cv_results(300,3000,df3)
print_cv_results(300,3000,df4)
print_cv_results(300,3000,df5)

rf_300n_3000m_hasilsmote_1
AUC : [0.98465249]
Accuracy : [0.94486054]
F-measure : [0.9392457]
Precision : [0.95760218]
Recall : [0.93052686]
rf_300n_3000m_hasilsmote_2
AUC : [0.98783282]
Accuracy : [0.95015496]
F-measure : [0.94547549]
Precision : [0.9651667]
Recall : [0.93414256]
rf_300n_3000m_hasilsmote_3
AUC : [0.98369953]
Accuracy : [0.94615186]
F-measure : [0.94145933]
Precision : [0.95727016]
Recall : [0.93362603]
rf_300n_3000m_hasilsmote_4
AUC : [0.98543815]
Accuracy : [0.94757231]
F-measure : [0.94210298]
Precision : [0.96259086]
Recall : [0.93130165]
rf_300n_3000m_hasilsmote_5
AUC : [0.98479568]
Accuracy : [0.94356921]
F-measure : [0.93776124]
Precision : [0.96070927]
Recall : [0.92639463]


In [9]:
rf1 = pickle.load(open('../data3/model/rf_500n_3000m_hasilsmote_1', 'rb'))
rf2 = pickle.load(open('../data3/model/rf_100n_3000m_hasilsmote_2', 'rb'))
rf3 = pickle.load(open('../data3/model/rf_500n_2000m_hasilsmote_3', 'rb'))
rf4 = pickle.load(open('../data3/model/rf_500n_3000m_hasilsmote_4', 'rb'))
rf5 = pickle.load(open('../data3/model/rf_500n_2000m_hasilsmote_5', 'rb'))

In [8]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [19]:
df.X.tail()

,1,2,3,4,5,6,7,8,9,10,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281
4552,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.138955,0.102138,0.140143,0.000000,0.047506,0.085511,0.349169,0.000000,0.264846,0.076010
4553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.184748,0.190118,0.122449,0.085929,0.058002,0.157895,0.367347,0.239527,0.137487
4554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.022210,0.088155,0.067198,0.051936,0.031891,0.053303,0.223235,0.493394,0.025399,0.065604
4555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124204,0.000000,0.187898,0.181529,0.063694,0.000000,0.312102,0.363057,0.000000,0.203822
4556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.085060,0.140676,0.064340,0.124318,0.130861,0.137405,0.748092,0.248637,0.121592,0.139586


In [24]:
#ens_grid={'voting':['soft','hard']}
#[rf1,rf2,rf3,rf4,rf5]
ens = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3),('rf4', rf4), ('rf5', rf5)], voting='hard')# refit=False)
#ens = GridSearchCV(ens, ens_grid, n_jobs=-1, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
ens=ens.fit(df.X,df.Y)

In [ ]:
y_predict1 = ens.predict(dtf.x)
tn1,fn1,fp1,tp1=confusion_matrix(dtf.y, y_predict1).ravel()
print (tn1,fn1)
print (fp1,tp1)

In [ ]:
y_predict2 = ens.predict(dtl.x)
tn2,fn2,fp2,tp2=confusion_matrix(dtl.y, y_predict2).ravel()
print (tn2,fn2)
print (fp2,tp2)

In [ ]:
from sklearn import metrics

In [ ]:
akurasi1= metrics.accuracy_score(dtf.y, y_predict1)
auc1 = metrics.roc_auc_score(dtf.y, y_predict1)
presisi1 =metrics.precision_score(dtf.y, y_predict1)
recall1 =metrics.recall_score(dtf.y, y_predict1)
f11 =metrics.f1_score(dtf.y, y_predict1)
print("acc:",akurasi1)
print("auc:",auc1)
print("prec:",presisi1)
print("recc:",recall1)
print("f1:",f11)

In [ ]:
akurasi2= metrics.accuracy_score(dtl.y, y_predict1)
auc2 = metrics.roc_auc_score(dtl.y, y_predict1)
presisi2 =metrics.precision_score(dtl.y, y_predict1)
recall2 =metrics.recall_score(dtl.y, y_predict1)
f12 =metrics.f1_score(dtl.y, y_predict1)
print("acc:",akurasi2)
print("auc:",auc2)
print("prec:",presisi2)
print("recc:",recall2)
print("f1:",f12)

In [39]:
#ens_grid={'voting':['soft','hard']}
#[rf1,rf2,rf3,rf4,rf5]
ens2 = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3),('rf4', rf4), ('rf5', rf5)], voting='soft')# refit=False)
#ens = GridSearchCV(ens, ens_grid, n_jobs=-1, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
ens2=ens2.fit(df.X,df.Y)

In [40]:
y_predict1 = ens2.predict(dtf.x)
tn1,fn1,fp1,tp1=confusion_matrix(dtf.y, y_predict1).ravel()
print (tn1,fn1)
print (fp1,tp1)

105 135
0 0


In [41]:
y_predict2 = ens2.predict(dtl.x)
tn2,fn2,fp2,tp2=confusion_matrix(dtl.y, y_predict2).ravel()
print (tn2,fn2)
print (fp2,tp2)

1484 1564
0 0


In [42]:
from sklearn import metrics

In [43]:
akurasi1= metrics.accuracy_score(dtf.y, y_predict1)
auc1 = metrics.roc_auc_score(dtf.y, y_predict1)
presisi1 =metrics.precision_score(dtf.y, y_predict1)
recall1 =metrics.recall_score(dtf.y, y_predict1)
f11 =metrics.f1_score(dtf.y, y_predict1)
print("acc:",akurasi1)
print("auc:",auc1)
print("prec:",presisi1)
print("recc:",recall1)
print("f1:",f11)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
akurasi2= metrics.accuracy_score(dtl.y, y_predict1)
auc2 = metrics.roc_auc_score(dtl.y, y_predict1)
presisi2 =metrics.precision_score(dtl.y, y_predict1)
recall2 =metrics.recall_score(dtl.y, y_predict1)
f12 =metrics.f1_score(dtl.y, y_predict1)
print("acc:",akurasi2)
print("auc:",auc2)
print("prec:",presisi2)
print("recc:",recall2)
print("f1:",f12)

In [ ]:
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

Y_COLUMN = "author"
TEXT_COLUMN = "text"


classifiers = [
    ('rf1', rf1), ('rf2', rf2), ('rf3', rf3),('rf4', rf4), ('rf5', rf5),
]

mixed_pipe = Pipeline([
    ("voting", VotingClassifier(classifiers, voting="soft"))
])

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = dict(
    voting__weights=[1,1,1,1]
)


grid_search = GridSearchCV(mixed_pipe, param_grid=param_grid, n_jobs=-1, verbose=10, scoring="neg_log_loss")

grid_search.fit(df.X, df.Y)

cv_results = grid_search.cv_results_

for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(params, mean_score)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))